In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.layers import BatchNormalization,Activation,Add,Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D,MaxPooling2D
from tensorflow.python.keras.optimizers import adam_v2
from tensorflow.keras.optimizers import SGD,Nadam
from tensorflow.python.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
tf.__version__

'2.4.1'

In [2]:
df = open('./CSVs/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

#數據生成器:用來训练神经网络时逐批次地加载数据
#---------------------------------------------------------------------------
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, games, batch_size):
        self.games = games
        self.batch_size = batch_size
        self.index = 0

    def __len__(self):
        return sum(len(game.split(',')) for game in self.games) // self.batch_size

    def __getitem__(self, index):
        x_batch, y_batch = [], []
        while len(x_batch) < self.batch_size:
            game = self.games[self.index]
            moves_list = game.split(',')
            for count, move in enumerate(moves_list):
                if len(x_batch) < self.batch_size:
                    x_batch.append(prepare_input(moves_list[:count]))
                    y_batch.append(prepare_label(moves_list[count]))
                else:
                    break  # Break when the batch is full
            self.index = (self.index + 1) % len(self.games)  # Move to the next game

        x_batch = np.array(x_batch)
        y_batch = np.array(y_batch)
        y_batch_one_hot = tf.one_hot(y_batch, depth=19*19)

        return x_batch, y_batch_one_hot

    def on_epoch_end(self):
        np.random.shuffle(self.games)
#---------------------------------------------------------------------------

# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 118500, Total Moves: 27135638


In [3]:
def Conv_BN_Relu(filters, kernel_size, strides, input_layer):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# ResNet18网络对应的残差模块a和残差模块b
def resiidual_a_or_b(input_x, filters, flag):
    #残差模块a
    if flag == 'a':
        # 主路
        x = Conv_BN_Relu(filters, (3, 3), 1, input_x)
        x = Conv_BN_Relu(filters, (3, 3), 1, x)
        # 输出
        y = Add()([x, input_x])
        return y
    #残差模块b
    elif flag == 'b':
        # 主路
        x = Conv_BN_Relu(filters, (3, 3), 2, input_x)
        x = Conv_BN_Relu(filters, (3, 3), 1, x)
        # 支路下采样
        input_x = Conv_BN_Relu(filters, (1, 1), 2, input_x)
        # 输出
        y = Add()([x, input_x])
        return y
def create_model():
    # 第一层
    input_layer = Input((19, 19, 4))
    x = Conv_BN_Relu(128,(3, 3), 1, input_layer)
    x = Conv_BN_Relu(128,(3, 3), 1, x)
    
    # conv2_x
    x = resiidual_a_or_b(x, 256, 'b')
    x = Dropout(0.2)(x)
    x = resiidual_a_or_b(x, 256, 'a')

    #conv3_x
    x = resiidual_a_or_b(x, 512, 'b')
    x = Dropout(0.2)(x)
    x = resiidual_a_or_b(x, 512, 'a')


    # 最后一层
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(600)(x)
    x = Dropout(0.5)(x)
    x = Dense(400)(x)
    x = Dropout(0.5)(x)
    x = Dense(361)(x)
    y = Softmax(axis=-1)(x)
    model = Model([input_layer], [y])


    optimizer = 'adam'
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model
model = create_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 19, 19, 4)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 19, 19, 128)  4736        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 19, 19, 128)  512         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 19, 19, 128)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [4]:
start = 1
end = 30000
batch_size = 1024
train_losses = []
val_losses = []
train_accuracy = []
val_accuracy = []

# 尋找所有文件
#saved_models = os.listdir('./results/Kyu/')
# 找到具有最低验证损失的模型文件
#best_model = min(saved_models, key=lambda x: float(x.split('_')[2]) if x.startswith('kyu_loss-accuracy') else float('inf'))
# 构建最佳模型的路径
#best_model_path = os.path.join('./results/Kyu/', best_model)
#print("加载模型:",best_model_path)
# 加载最佳模型的权重
#model.load_weights(best_model_path)

# 定义 EarlyStopping 回调 :回调参数
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for i in range(3):
    print("开始训练:", start, "-", end)

    # 定义 ModelCheckpoint 回调 :回调保存
    checkpoint = ModelCheckpoint(
        f'./results/Kyu/kyu_loss-accuracy_{{val_loss:.4f}}_{{val_accuracy:.4f}}_{start}_{end}.h5',
        monitor='val_loss',
        save_best_only=True)

    # 將數據分成訓練和驗證集
    games_train, games_val = train_test_split(games[start:end], test_size=0.10,random_state=42)

    # 創建數據生成器實例
    train_generator = DataGenerator(games_train, batch_size)
    val_generator = DataGenerator(games_val, batch_size)



    #用数据生成器进行训练
    history = model.fit(
        x=train_generator,
        epochs=100,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stopping],
        shuffle=True
    )


    # 尋找所有文件
    saved_models = os.listdir('./results/Kyu/')
    # 找到具有最低验证损失的模型文件
    best_model = min(saved_models, key=lambda x: float(x.split('_')[2]) if x.startswith('kyu_loss-accuracy') else float('inf'))
    # 构建最佳模型的路径
    best_model_path = os.path.join('./results/Kyu/', best_model)
    print("加载模型:",best_model_path)
    # 加载最佳模型的权重
    model.load_weights(best_model_path)




    #將每個 epoch 的值添加到列表中
    train_losses.extend(history.history['loss'])
    val_losses.extend(history.history['val_loss'])
    train_accuracy.extend(history.history['accuracy'])
    val_accuracy.extend(history.history['val_accuracy'])


    start=start+30000;
    end=end+30000;
    if (end==90000):
        end=110000

开始训练: 1 - 30000
Epoch 1/100
6056/6056 [==============================] - 2671s 440ms/step - loss: 3.7057 - accuracy: 0.2702 - val_loss: 2.5967 - val_accuracy: 0.4494
Epoch 2/100
6056/6056 [==============================] - 2663s 440ms/step - loss: 2.6654 - accuracy: 0.4442 - val_loss: 2.4632 - val_accuracy: 0.4650
Epoch 3/100
6056/6056 [==============================] - 2659s 439ms/step - loss: 2.5245 - accuracy: 0.4607 - val_loss: 2.3416 - val_accuracy: 0.4776
Epoch 4/100
6056/6056 [==============================] - 2658s 439ms/step - loss: 2.4433 - accuracy: 0.4712 - val_loss: 2.3015 - val_accuracy: 0.4871
Epoch 5/100
6056/6056 [==============================] - 2660s 439ms/step - loss: 2.3876 - accuracy: 0.4784 - val_loss: 2.2739 - val_accuracy: 0.4891
Epoch 6/100
6056/6056 [==============================] - 2661s 439ms/step - loss: 2.3434 - accuracy: 0.4842 - val_loss: 2.2483 - val_accuracy: 0.4911
Epoch 7/100
6056/6056 [==============================] - 2660s 439ms/step - loss: 2.

KeyboardInterrupt: 

# 

# 這裡內容為訓練中斷,需要重新讀取權重,繼續訓練

# 

In [4]:
'''
start = 60001
end = 110000
batch_size = 1024
train_losses = []
val_losses = []
train_accuracy = []
val_accuracy = []

# 尋找所有文件
saved_models = os.listdir('./results/Kyu/')
# 找到具有最低验证损失的模型文件
best_model = min(saved_models, key=lambda x: float(x.split('_')[2]) if x.startswith('kyu_loss-accuracy') else float('inf'))
# 构建最佳模型的路径
best_model_path = os.path.join('./results/Kyu/', best_model)
print("加载模型:",best_model_path)
# 加载最佳模型的权重
model.load_weights(best_model_path)

# 定义 EarlyStopping 回调 :回调参数
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

for i in range(1):
    print("开始训练:", start, "-", end)

    # 定义 ModelCheckpoint 回调 :回调保存
    checkpoint = ModelCheckpoint(
        f'./results/Kyu/kyu_loss-accuracy_{{val_loss:.4f}}_{{val_accuracy:.4f}}_{start}_{end}.h5',
        monitor='val_loss',
        save_best_only=True)

    # 將數據分成訓練和驗證集
    games_train, games_val = train_test_split(games[start:end], test_size=0.10,random_state=42)

    # 創建數據生成器實例
    train_generator = DataGenerator(games_train, batch_size)
    val_generator = DataGenerator(games_val, batch_size)



    #用数据生成器进行训练
    history = model.fit(
        x=train_generator,
        epochs=100,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stopping,
                  ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)],
        shuffle=True
    )


    # 尋找所有文件
    saved_models = os.listdir('./results/Kyu/')
    # 找到具有最低验证损失的模型文件
    best_model = min(saved_models, key=lambda x: float(x.split('_')[2]) if x.startswith('kyu_loss-accuracy') else float('inf'))
    # 构建最佳模型的路径
    best_model_path = os.path.join('./results/Kyu/', best_model)
    print("加载模型:",best_model_path)
    # 加载最佳模型的权重
    model.load_weights(best_model_path)




    #將每個 epoch 的值添加到列表中
    train_losses.extend(history.history['loss'])
    val_losses.extend(history.history['val_loss'])
    train_accuracy.extend(history.history['accuracy'])
    val_accuracy.extend(history.history['val_accuracy'])


    start=start+30000;
    end=end+30000;
    if (end==90000):
        end=110000
'''

加载模型: results/kyu_loss-accuracy_2.0823_0.5143_60001_110000.h5
开始训练: 60001 - 110000
Epoch 1/100
10009/10009 [==============================] - 4458s 445ms/step - loss: 2.1358 - accuracy: 0.5128 - val_loss: 2.0857 - val_accuracy: 0.5148
Epoch 2/100
10009/10009 [==============================] - 4427s 442ms/step - loss: 2.1242 - accuracy: 0.5144 - val_loss: 2.0860 - val_accuracy: 0.5162
Epoch 3/100
10009/10009 [==============================] - 4427s 442ms/step - loss: 2.1082 - accuracy: 0.5171 - val_loss: 2.0816 - val_accuracy: 0.5151
Epoch 4/100
10009/10009 [==============================] - 4416s 441ms/step - loss: 2.0924 - accuracy: 0.5192 - val_loss: 2.0896 - val_accuracy: 0.5160
Epoch 5/100
10009/10009 [==============================] - 4442s 444ms/step - loss: 2.0819 - accuracy: 0.5212 - val_loss: 2.0814 - val_accuracy: 0.5153
Epoch 6/100
10009/10009 [==============================] - 4450s 445ms/step - loss: 2.0662 - accuracy: 0.5234 - val_loss: 2.0833 - val_accuracy: 0.5160
Epoch

KeyboardInterrupt: 